In [1]:
import os, sys
parent_folder_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_folder_path)

In [2]:
import pandas as pd

data_df = pd.read_pickle('../costa_rica/data/Costa_rica_X.pkl')
data_df.head()

,bookmark_count,bookmarked,conversation_id_str,created_at,display_text_range,entities,favorite_count,favorited,full_text,is_quote_status,...,MW_Replies,MW_Reposts,MW_Comments,MW_Reactions,MW_Views,MW_Estimated Views,MW_Document Tags,post_type,url,has_retweeted_status_result
3,0,False,1816217080397455588,Wed Jul 24 21:01:10 +0000 2024,"[0, 276]","{'hashtags': [], 'symbols': [], 'timestamps': ...",3,False,Algunos padres están alarmados por nuevas circ...,False,...,NaN,NaN,NaN,3.0,15.0,NaN,NaN,Post,https://twitter.com/1302389542457159685/status...,False
5,0,False,1816201806252490955,Wed Jul 24 20:39:44 +0000 2024,"[5, 72]","{'hashtags': [], 'symbols': [], 'timestamps': ...",0,False,@T13 .\nTodos extranjeros no pago vacuna algun...,False,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Comment,https://twitter.com/15626578/status/1816211686...,False
6,0,False,1815161737496134018,Wed Jul 24 20:11:18 +0000 2024,"[53, 171]","{'hashtags': [], 'symbols': [], 'timestamps': ...",1,False,@Luis314PII @linea_progre @tasty4ever1 @esCesa...,False,...,1.0,NaN,NaN,NaN,9.0,NaN,NaN,Comment,https://twitter.com/78798396/status/1816204532...,False
7,0,False,1815161737496134018,Wed Jul 24 20:10:29 +0000 2024,"[53, 305]","{'hashtags': [], 'symbols': [], 'timestamps': ...",1,False,@Luis314PII @linea_progre @tasty4ever1 @esCesa...,False,...,NaN,NaN,NaN,1.0,4.0,NaN,NaN,Comment,https://twitter.com/78798396/status/1816204327...,False
11,0,False,1816165237101662691,Wed Jul 24 17:35:09 +0000 2024,"[0, 280]","{'hashtags': [], 'symbols': [], 'timestamps': ...",13,False,Estimados compañeros Bicentenarios.\nEste es u...,False,...,NaN,7.0,NaN,13.0,102.0,NaN,NaN,Post,https://twitter.com/106178430/status/181616523...,False


In [3]:
data_df.columns

Index(['bookmark_count', 'bookmarked', 'conversation_id_str', 'created_at',
       'display_text_range', 'entities', 'favorite_count', 'favorited',
       'full_text', 'is_quote_status', 'lang', 'quote_count', 'reply_count',
       'retweet_count', 'retweeted', 'retweeted_status_result', 'user_id_str',
       'status_id', 'extended_entities', 'possibly_sensitive',
       'possibly_sensitive_editable', 'in_reply_to_screen_name',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'note_tweet',
       'quoted_status_id_str', 'place', 'conversation_control', 'MW_Date',
       'MW_Time', 'MW_Document ID', 'MW_URL', 'MW_Input Name', 'MW_Keywords',
       'MW_Information Type', 'MW_Source Type', 'MW_Source Name',
       'MW_Source Domain', 'MW_Content Type', 'MW_Author Name',
       'MW_Author Handle', 'MW_Title', 'MW_Opening Text', 'MW_Hit Sentence',
       'MW_Image', 'MW_Hashtags', 'MW_Links', 'MW_Country', 'MW_Region',
       'MW_State', 'MW_City', 'MW_Language', 'MW_Sentiment

In [7]:
data_df.has_retweeted_status_result[:5]

3     False
5     False
6     False
7     False
11    False
Name: has_retweeted_status_result, dtype: bool

In [8]:
print('Total rows', data_df.shape[0])
print('Empty text rows', data_df.full_text.isnull().sum())
print('Unique text rows', data_df.full_text.nunique())

Total rows 5444
Empty text rows 0
Unique text rows 3486


In [9]:
## save pickle with cleaned data
exp1_df = data_df.drop_duplicates(subset='full_text')
print('Unique rows', exp1_df.shape[0])


Unique rows 3486


In [7]:
# save expiriment data
exp1_df.to_pickle('../data/costa_rica_exp1.pkl')

In [19]:
import os
from dotenv import load_dotenv
from openai import OpenAI as OpenAIclient
from bertopic.backend import OpenAIBackend
from utils.embedding import precompute_embeddings

load_dotenv()

client = OpenAIclient(api_key=os.getenv('OPENAI_API_KEY'))
embedding_model = OpenAIBackend(client, 'text-embedding-3-small', batch_size=1000)
embedding_function = embedding_model.embed

# precompute embeddings
embeddings = precompute_embeddings(data_df = exp1_df,
                                   text_col_name='full_text',
                                   embedding_function=embedding_function,
                                   output_file_path='../data/costa_rica_exp1_embeddings.npy')

In [10]:
from utils.embedding import precompute_embeddings, read_embeddings

embeddings = read_embeddings('../data/costa_rica_exp1_embeddings.npy')
embeddings[:2]

array([[ 0.04628968,  0.01328233,  0.00646305, ...,  0.03969432,
        -0.00095864,  0.01252916],
       [ 0.02764316, -0.01576809,  0.01681836, ...,  0.02716704,
         0.0169724 ,  0.00551743]])

In [13]:
from bertopic import BERTopic
from nltk.corpus import stopwords
from openai import OpenAI as OpenAIclient
from bertopic.backend import OpenAIBackend
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, OpenAI, MaximalMarginalRelevance

docs = exp1_df.full_text.to_list()

# hyperparameters from coherence and diversity based experimentation
params_list = [{'nr_topics': 15, 'min_topic_size': 15},
               {'nr_topics': 15, 'min_topic_size': 50},
               {'nr_topics': 15, 'min_topic_size': 100},
               {'nr_topics': 50, 'min_topic_size': 10},
               {'nr_topics': 100, 'min_topic_size': 100}]

# lists to save the two resulting models
models = []
topics = []
probs = []

# initialize custom model components
client = OpenAIclient(api_key=os.getenv('OPENAI_API_KEY'))
embedding_model = OpenAIBackend(client, 'text-embedding-3-small', batch_size=1000)
stop_words = stopwords.words(['english', 'spanish']) + ['https', 'http', 'www', 'com', 'org', 'net', 'edu', 'gov']
vectorizer_model = CountVectorizer(ngram_range=(1, 2),stop_words=stop_words)
main_representation = OpenAI(client, model='gpt-4o-2024-05-13', delay_in_seconds=5, chat=True)
keyword_representation = KeyBERTInspired()
keyword_sent_representation = [KeyBERTInspired(top_n_words=30), OpenAI(client, model='gpt-4o-2024-05-13', delay_in_seconds=10, chat=True)]


representation_model = {
   "Main": main_representation,
   "Keywords":  keyword_representation,
   "MMR Keywords":  keyword_sent_representation,
}

for params in params_list:

    model = BERTopic(
        embedding_model = embedding_model,
        vectorizer_model = vectorizer_model,
        representation_model = representation_model,
        nr_topics = params['nr_topics'],
        min_topic_size = params['min_topic_size'],
        calculate_probabilities = True,
        verbose = True
    )

    topic, prob = model.fit_transform(documents=docs, embeddings=embeddings)

    print(f'nr_topics:       {params["nr_topics"]}')
    print(f'min_topic_size:  {params["min_topic_size"]}')
    print(f'num_outliers:    {model.get_topic_info(-1)["Count"][0]}')

    # save results
    models.append(model)
    topics.append(topic)
    probs.append(prob)

2024-08-01 09:34:24,554 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


2024-08-01 09:34:39,613 - BERTopic - Dimensionality - Completed ✓
2024-08-01 09:34:39,614 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-01 09:34:39,871 - BERTopic - Cluster - Completed ✓
2024-08-01 09:34:39,872 - BERTopic - Representation - Extracting topics from clusters using representation models.
  0%|          | 0/36 [00:07<?, ?it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [51]:
# pd.set_option('max_colwidth', 800)

for idx,model in enumerate(models):
    model.get_topic_info()[['Topic', 'Count', 'Name', 'Keywords']].to_csv(f'/Users/wesamazaizeh/Documents/pfizer_slt/costa_rica/resultstopics_model_v2_{idx}.csv', index=False)

In [52]:
models[0].visualize_topics()

In [31]:
hierarchical_topics = models[4].hierarchical_topics(docs)
tree_1 = models[4].visualize_hierarchy(hierarchical_topics=hierarchical_topics)
tree_1.show()
try:
    tree_1.write_html('/Users/wesamazaizeh/Documents/pfizer_slt/costa_rica/resultshierarchical_tree_5.html')
except FileNotFoundError:
    os.mkdir('../data/costa_rica_exp1')
    tree_1.write_html('/Users/wesamazaizeh/Documents/pfizer_slt/costa_rica/resultshierarchical_tree_1.html')

100%|██████████| 4/4 [00:04<00:00,  1.20s/it]
